<a href="https://colab.research.google.com/github/L4ncelot1024/Learn_Deep_Learning_Le_Wagon/blob/main/Day2/01_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

## Introduction

In this exercise, you will do text classification starting from raw text (as
a set of text files on disk). You will apply this workflow on the IMDB sentiment
classification dataset (unprocessed version) where the goal is to say if a review is positive or negative.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.4.1


## Data Preparation

This first part is a good example of the beginning of any Deep Learning project: first we need to look at the data and prepare them ! 

Our data here are movie reviews written by users on the [imdb website](https://www.imdb.com/). We download them already labelled, the positive and negative ones are split in two different folders. 

But the data are not processed yet so we will use tensorflow to prepare the data.

### Load the data: IMDB movie review sentiment classification

Let's download the data and inspect its structure.

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  25.5M      0  0:00:03  0:00:03 --:--:-- 25.5M


The `aclImdb` folder contains a `train` and `test` subfolder. The following bash commands prints the content of the different folder

In [3]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!cat aclImdb/README

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

In [5]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


The `aclImdb/train/pos` and `aclImdb/train/neg` folders contain text files, each file represents one review (**positive** if in the *pos* folder or **negative** if in the *neg* folder). The following command display one example from each folder

In [7]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [8]:
!cat aclImdb/test/neg/62_2.txt

Humm, an Italian movie starred by David hasselhoff and Linda Blair, I wasn´t expecting very much, to be honest and in fact, I took even less than I was expecting. It doesn´t mean this movie is the worst I have seen because I have watched worse things than this but the plot was most of the times confusing and uninteresting and some good gore scenes are the only thing saving this. Apart from that you are going to love some special effects, they are really cheesy and bad. Now I only want to watch "Troll 3" by this same director, sure it is not going to be worse than that.

We are only interested in the `pos` and `neg` subfolders, so let's delete the rest:

In [9]:
!rm -r aclImdb/train/unsup

Now we would like to load our data from the files into the memory so we can feed them into our model. 

To do that, use the utility `tf.keras.preprocessing.text_dataset_from_directory` to
generate a labeled `tf.data.Dataset` object from a set of text files on disk filed into class-specific folders. 

We will also split the data since this function already contains the logic to split a dataset into validation and test (using the two argument `validation_split` and `subset`).

Your goal here is to use this function to generate the training, validation, and test datasets. 

The validation and training datasets are generated from two subsets of the `train` directory, with the following ratio: 80% in the training and 20% in the validation. The test dataset comes directly from the test folder. 

You can use a reasonable batch size (for instance `32`). 

Finishes with a print of the sizes of your 3 sets. 

NB: When using the `validation_split` & `subset` arguments, make sure to either specify a random seed, or to pass `shuffle=False`, so that the validation & training splits you get have no overlap.

In [10]:
# YOUR CODE HERE
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',batch_size=32,shuffle=True,seed=42,validation_split=0.2,subset='training')
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',batch_size=32,shuffle=True,seed=42,validation_split=0.2,subset='validation')
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/test',batch_size=32,shuffle=True,seed=42)
print(len(raw_train_ds),len(raw_val_ds),len(raw_test_ds))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
625 157 782



<details>
  <summary>View solution</summary>
    
```python
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)
```
</details>



Let's preview a few samples:

In [11]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
0
b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into complicated situatio

Important: Your dataset is not prepared yet, you just explained how to load it !

### Prepare the data

From above, you see there are some noise in the data, for instance the `<br />` tags. So we want to remove them !

Having looked at our data above, we see that the raw text contains HTML break
tags of the form `<br />`. 

These tags will not be removed by the default standardizer (which doesn't strip HTML). Because of this, we will need to create a custom standardization function.

Write a function `custom_standardisation` which take text as input, removes the tags of the form  `<br />` and proceeds to tradional standardisation: lowering caracters, removing punctuation. 

We recommend using the internal tensorflow methods inside `tf.strings`: `tf.strings.lower`, `tf.strings.regex_replace`, `string/punctuation`

In [12]:
import string
import re

def custom_standardization(input_data):
  # YOUR CODE HERE
  output_data = tf.strings.lower(input_data)
  output_data = tf.strings.regex_replace(output_data,'<br />',' ',replace_global=True)
  output_data = tf.strings.regex_replace(output_data, "[%s]" % re.escape(string.punctuation), "")
  return output_data

<details>
  <summary>View solution</summary>
    
```python
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )
```
</details>



Now we will use this function in a vectoriation workflow using `TextVectorization`. This helper will normalize, split, and map strings to integers. 

We suggest default values for the model constants.

In [13]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Model constants.
max_features = 20000
sequence_length = 80

# YOUR CODE HERE
vectorize_layer = TextVectorization(max_tokens=max_features, standardize=custom_standardization, output_mode='int', output_sequence_length=sequence_length)

<details>
  <summary>Hint</summary>
    
```markdown
Have  loot at the documentation [TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization)
```
    
</details>
​
<details>
  <summary>View solution</summary>
    
```python
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

```
</details>

In [14]:
# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)


In [15]:
#chck text_ds
for text_batch in text_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])


b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />There\'s a mysteri

### Two options to vectorize the data

There are 2 ways we can use our text vectorization layer:

**Option 1: Make it part of the model**, so as to obtain a model that processes raw
 strings, like this:

```python
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
...
```

**Option 2: Apply it to the text dataset** to obtain a dataset of word indices, then
 feed it into a model that expects integer sequences as inputs.

An important difference between the two methods is that  option 2 enables you to do
**asynchronous CPU processing and buffering** of your data when training your model on the graphical processing unit, a.k.a GPU (this is the hardware on which a neural network is trained). This is the case since you run first your pre-processing on all the data and then train your model, so the first step can occur on the CPU, where it will be more efficient.


Here since we're training the model on GPU, we want to go with this 2nd option to get the best performance. This is what we will do below.

If we were to export our model to production, we'd ship a model that accepts raw
strings as input so we would use the first option. But we can actually recover this first option from a model trained with the second one. We do this in the last section.

Write a function you will apply to your 3 datasets (`raw_train_ds`, `raw_val_ds` and `raw_test_ds`) to vectorize them. Recall that each dataset contains tuple of `(text, label)`

In [16]:
# This function should return 2 elements: the text vectorized and the label
def vectorize_text(text, label):
    # YOUR CODE HERE
    vectorized = vectorize_layer(tf.expand_dims(text, -1))
    return vectorized, label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)




<details>
  <summary>View solution</summary>
    
```python
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

```
</details>

In [17]:
#aquestion : a quoi ca correspond ?
# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [18]:
#why label 0 ?
for text_batch, label_batch in train_ds.take(3):
      print(text_batch.numpy()[0])
      print(label_batch.numpy()[0])

[   86    17   260     2   222     1   571    31   229    11  2418 10253
    51    22    25   404   251    12   306   282     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
0
[  10   13  747   51   10  208   11   19  450  548    9   13    2  113
  121  814    5    2  644   87  665   10   13   87   97  274 1007 6768
  325   68 1059   11 1177    7    4  743    6   54  511    2 9245   25
  820    6 1222    8 1768    5  645 1246   60   23 2099   31    2  113
    5  206   18   77 2419    2  232    5    2  840   14   15    2  173
 7644 5372  158    4  239 2056 1881   18   92   97]
0
[  31   83   10  197    2 1675   59   26    4   50   70 1210   17   16
   29 1826  111   87  747   13

## Model

## Architecture

First, we choose a very simple architecture for our baseline model:
- `Embedding` layer to create embedding for our entries;
- 1 `LSTM` layer;
- 1 `Dense` layer for the final prediction.

We will use the `adam` optimizer. The task is to classify our entries between two classes, what should be the size of the final `Dense` layer and which loss will you use?

Then you can try out more advanced architectures at the end!

In [19]:
#see arguments of LSTM
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout


def build_baseline_model(embedding_dim = 128):
  model = Sequential()

  # YOUR CODE HERE
  # add your layers
  model.add(Embedding(max_features,embedding_dim))
  model.add(LSTM(128, dropout=0.2))
  model.add(Dense(1,activation='sigmoid'))

  # YOUR CODE HERE
  # Compile the model 
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model



<details>
  <summary>View solution</summary>
    
```python
def build_baseline_model(embedding_dim=32, conv_dim=128, num_filters=7):
  model = Sequential()

  # Next, we add a layer to map those vocab indices into a space of dimensionality
  # 'embedding_dim'.
  model.add(Embedding(max_features, embedding_dim))
  model.add(LSTM(128, dropout=0.2))
  model.add(Dense(1, activation="sigmoid"))

  # Compile the model with binary crossentropy loss and an adam optimizer.
  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  return model
```
</details>

## Train the model

In [20]:

print(train_ds,val_ds)
for example in val_ds.take(2):
  print(example[0].numpy())
  print(example[1].numpy())
  exit()


<PrefetchDataset shapes: ((None, 80), (None,)), types: (tf.int64, tf.int32)> <PrefetchDataset shapes: ((None, 80), (None,)), types: (tf.int64, tf.int32)>
[[   4  469  495 ...  468    5  918]
 [   2  136   23 ...    0    0    0]
 [ 254   10  237 ...  262    4  171]
 ...
 [   4   52 4868 ...  322 1176   51]
 [   4  590    5 ...  289    9  718]
 [  10  606    6 ...    3   24  116]]
[0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1]
[[2820 1598    7 ...  178    4  356]
 [  11   17   43 ... 2555 8096  120]
 [  10  495   11 ...  443   20  971]
 ...
 [  11 1264  757 ... 4724    1    3]
 [  11   19   43 ...    3 1223   50]
 [ 218   14 2225 ...  182    5    2]]
[0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0]


In [21]:
#ca plante !!
epochs = 9

model = build_baseline_model()
# Fit the model using the train and test datasets.
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/9
625/625 [==============================] - 57s 40ms/step - loss: 0.5749 - accuracy: 0.6800 - val_loss: 0.4512 - val_accuracy: 0.7986
Epoch 2/9
625/625 [==============================] - 20s 32ms/step - loss: 0.3283 - accuracy: 0.8673 - val_loss: 0.4861 - val_accuracy: 0.8116
Epoch 3/9
625/625 [==============================] - 19s 31ms/step - loss: 0.2117 - accuracy: 0.9212 - val_loss: 0.5478 - val_accuracy: 0.7988
Epoch 4/9
625/625 [==============================] - 19s 31ms/step - loss: 0.1405 - accuracy: 0.9508 - val_loss: 0.6940 - val_accuracy: 0.7786
Epoch 5/9
625/625 [==============================] - 19s 31ms/step - loss: 0.1199 - accuracy: 0.9581 - val_loss: 0.7801 - val_accuracy: 0.7990
Epoch 6/9
625/625 [==============================] - 19s 31ms/step - loss: 0.0696 - accuracy: 0.9772 - val_loss: 0.9482 - val_accuracy: 0.7972
Epoch 7/9
625/625 [==============================] - 19s 31ms/step - loss: 0.0630 - accuracy: 0.9793 - val_loss: 0.9347 - val_accuracy: 0.7994

## Evaluate the model on the test set

In [22]:
def plot_history(history):
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.legend()
    plt.show()

In [2]:
# Plot history
plot_history(history)

NameError: ignored

Apply your model on your test dataset and evaluate its predictions.

In [ ]:
evaluations = model.evaluate(test_ds)
test_evaluations = {name: value for value, name in zip(evaluations, model.metrics_names)}
print(test_evaluations)

196/196 [==============================] - 1s 5ms/step - loss: 0.9089 - accuracy: 0.7532
{'loss': 0.9089375138282776, 'accuracy': 0.7531999945640564}


<details>
  <summary>Hint</summary>
    
```markdown
Check the `model.evaluate` function documentation (how could you retrieve the name of the float returned?)
```
    
</details>
​
<details>
  <summary>View solution</summary>
    
```python
evaluations = model.evaluate(test_ds)
test_evaluations = {name: value for value, name in zip(evaluations, model.metrics_names)}
print(test_evaluations)

```
</details>

## Make an end-to-end model

If you want to obtain a model capable of processing raw strings, you can simply
create a new model, using the baseline you just defined and adding as a first step the vectorize layer. You can again do that with the Keras `Sequential` API

In [ ]:
end2end_model = tf.keras.Sequential()
end2end_model.add(tf.keras.Input(shape=(1,), dtype="string"))
end2end_model.add(vectorize_layer)
end2end_model.add(model)


end_to_end_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

196/196 [==============================] - 9s 41ms/step - loss: 0.9130 - accuracy: 0.7528


[0.9089372754096985, 0.7531999945640564]

<details>
  <summary>View solution</summary>
    
```python
end2end_model = tf.keras.models.Sequential()
# A string input
end2end_model.add(tf.keras.Input(shape=(1,), dtype="string"))
# Turn strings into vocab indices
end2end_model.add(vectorize_layer)
end2end_model.add(model)

end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

```
</details>

You can now apply this model on any input, so give it a try !

In [ ]:
# Create your own input
my_input_good = "I liked very much this movie"
my_input_good_2 = "This movie is not very bad though."
my_input_good_3 = "I fall asleep"
my_input_bad = "This movie is terribly bad"

# You just need to convert your string into a tensor
# YOUR CODE HERE
text_input = tf.convert_to_tensor([my_input_good, my_input_good_2, my_input_good_3, my_input_bad], dtype=tf.string)
end_to_end_model.predict(text_input)

<details>
  <summary>View solution</summary>
    
```python
text_input = tf.convert_to_tensor([my_input_good, my_input_good_2, my_input_good_3, my_input_bad], dtype=tf.string)
```
</details>

## Try out more complex architectures !

### Bi-LSTM

Here we suggest the following architecture:
- `Embedding` layer to create embedding for our entries;
- 1 or several `Bidirectional` layers (if multiple you need the first one to be a many-to-many one !)
- 1 Dense layer to add more complexity;
- 1 Dense layer for the outputs.

In [ ]:
from keras.layers import Bidirectional

def build_bi_lstm_model(embedding_dim=128, dense_dim=128):
  # YOUR CODE HERE
  return model

<details>
  <summary>View solution</summary>
    
```python
def build_bi_lstm_model(embedding_dim=128, dense_dim=128):
  # A integer input for vocab indices.
  model = Sequential()

  # Next, we add a layer to map those vocab indices into a space of dimensionality
  # 'embedding_dim'.
  model.add(Embedding(max_features, embedding_dim))
  model.add(Dropout(0.5))

  # Conv1D + global max pooling
  model.add(Bidirectional(layers.LSTM(64, return_sequences=True)))
  model.add(Bidirectional(layers.LSTM(64)))

  # We add a vanilla hidden layer:
  model.add(Dense(dense_dim, activation="relu"))
  model.add(Dropout(0.5))

  # We project onto a single unit output layer, and squash it with a sigmoid:
  model.add(Dense(1, activation="sigmoid", name="predictions"))

  # Compile the model with binary crossentropy loss and an adam optimizer.
  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  return model
```
</details>

In [ ]:
epochs = 9

bi_lstm_model = build_bi_lstm_model()
# Fit the model using the train and test datasets.
bi_lstm_history = bi_lstm_model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/9
157/157 [==============================] - 14s 57ms/step - loss: 0.6162 - accuracy: 0.6141 - val_loss: 0.3937 - val_accuracy: 0.8246
Epoch 2/9
157/157 [==============================] - 7s 46ms/step - loss: 0.3526 - accuracy: 0.8475 - val_loss: 0.4185 - val_accuracy: 0.8196
Epoch 3/9
157/157 [==============================] - 7s 45ms/step - loss: 0.2469 - accuracy: 0.9029 - val_loss: 0.4731 - val_accuracy: 0.8054
Epoch 4/9
157/157 [==============================] - 7s 45ms/step - loss: 0.1725 - accuracy: 0.9358 - val_loss: 0.5011 - val_accuracy: 0.8052
Epoch 5/9
157/157 [==============================] - 7s 45ms/step - loss: 0.1241 - accuracy: 0.9548 - val_loss: 0.7450 - val_accuracy: 0.7734
Epoch 6/9
157/157 [==============================] - 7s 46ms/step - loss: 0.0977 - accuracy: 0.9640 - val_loss: 0.7192 - val_accuracy: 0.8006
Epoch 7/9
157/157 [==============================] - 7s 46ms/step - loss: 0.0710 - accuracy: 0.9740 - val_loss: 0.8597 - val_accuracy: 0.8040
Epoch

### 1D-Convnet

Here we suggest the following architecture:
- `Embedding` layer to create embedding for our entries;
- eventually a Dropout after for regularization;
- 2 layers 1D `Conv1D`
- 1 GlobalMaxPooling1D layer to pool;
- 1 Dense layer for the outputs.

In [ ]:
from keras.layers import Conv1D, GlobalMaxPooling1D

def build_cnn_model(embedding_dim=128, conv_dim=128, num_filters=7):
  # YOUR CODE HERE
  return model

<details>
  <summary>View solution</summary>
    
```python
from keras.layers import Conv1D, GlobalMaxPooling1D

def build_cnn_model(embedding_dim=128, conv_dim=128, num_filters=7):
  # A integer input for vocab indices.
  model = Sequential()

  # Next, we add a layer to map those vocab indices into a space of dimensionality
  # 'embedding_dim'.
  model.add(Embedding(max_features, embedding_dim))
  model.add(Dropout(0.5))

  # Conv1D + global max pooling
  model.add(Conv1D(conv_dim, num_filters, padding="valid", activation="relu", strides=3))
  model.add(Conv1D(conv_dim, num_filters, padding="valid", activation="relu", strides=3))
  model.add(GlobalMaxPooling1D())

  # We add a vanilla hidden layer:
  model.add(Dense(conv_dim, activation="relu"))
  model.add(Dropout(0.5))

  # We project onto a single unit output layer, and squash it with a sigmoid:
  model.add(Dense(1, activation="sigmoid", name="predictions"))

  # Compile the model with binary crossentropy loss and an adam optimizer.
  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  return model
```
</details>

In [ ]:
epochs = 9

cnn_model = build_cnn_model()
# Fit the model using the train and test datasets.
cnn_history = cnn_model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/9
157/157 [==============================] - 6s 32ms/step - loss: 0.6861 - accuracy: 0.5211 - val_loss: 0.4645 - val_accuracy: 0.7806
Epoch 2/9
157/157 [==============================] - 5s 31ms/step - loss: 0.4473 - accuracy: 0.7964 - val_loss: 0.4323 - val_accuracy: 0.8102
Epoch 3/9
157/157 [==============================] - 5s 30ms/step - loss: 0.2618 - accuracy: 0.8996 - val_loss: 0.5236 - val_accuracy: 0.7990
Epoch 4/9
157/157 [==============================] - 5s 30ms/step - loss: 0.1451 - accuracy: 0.9495 - val_loss: 0.6559 - val_accuracy: 0.7948
Epoch 5/9
157/157 [==============================] - 5s 30ms/step - loss: 0.0952 - accuracy: 0.9660 - val_loss: 0.8134 - val_accuracy: 0.7830
Epoch 6/9
157/157 [==============================] - 5s 31ms/step - loss: 0.0773 - accuracy: 0.9716 - val_loss: 0.8962 - val_accuracy: 0.7988
Epoch 7/9
157/157 [==============================] - 5s 31ms/step - loss: 0.0497 - accuracy: 0.9830 - val_loss: 1.0438 - val_accuracy: 0.7940
Epoch 